In [44]:
using DataFrames, CSV, DelimitedFiles

In [45]:
NUM_CREWS = 10                
BREAK_LENGTH = 2       # how long at base to be considered "rested"

# tradeoffs
BETA = 100             # cost of one area unit burned / cost of mile traveled
ALPHA = 200            # cost of crew-day of suppression / cost of mile traveled
LINE_PER_CREW = 17     # how much perimeter prevented per crew per time period

17

In [46]:
# set path to all input data
in_path = "data/processed"

# get inital fire perimeters and no-suppression progression parameters
M = readdlm(in_path * "/sample_growth_patterns.csv", ',')
start_perims = M[:, 1]
progressions = M[:, 2:15]

NUM_TIME_PERIODS = size(M)[2] - 1 
NUM_FIRES = size(M)[1]       

# get distance from fire f to fire g 
fire_dists =  readdlm(in_path * "/fire_distances.csv", ',')

# get distance from base c to fire g (NUM_CREWS-by-NUM_FIRES)
base_fire_dists =  readdlm(in_path * "/base_fire_distances.csv", ',')

# initialize travel times (number of periods) from fire f to fire g
tau = convert(Array{Int}, ones(size(fire_dists)))

# initialize number of periods to travel from base c to fire g (NUM_CREWS-by-NUM_FIRES)
tau_base_to_fire = convert(Array{Int}, ones((size(base_fire_dists))))

# read intial crew statuses (location, period by which they must rest)
# (-1 in current_fire means crew is currently at base)
# (rested_periods is the amount of time crew has been at base, relevant for completing rest)
crew_starts = CSV.read(in_path * "/sample_crew_starts.csv", DataFrame)
println(size(crew_starts)[1])
rest_by = crew_starts[!, "rest_by"]
current_fire = crew_starts[!, "current_fire"]
rested_periods = crew_starts[!, "rested_periods"];

10


In [ ]:
# crew, from_type, from_ix, to_type, to_ix, from_time, to_time, from_rested, to_rested, exited_region

In [77]:
# get fire-to-fire arcs
ff = [[c, 1, f_from, 1, f_to, t_from, t_from + tau[f_to, f_from], rest, rest]
      for c=1:NUM_CREWS, f_from=1:NUM_FIRES, f_to=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
ff = copy(reduce(hcat, ff)');

# get fire-to-fire arcs from start, based on current crew locations
from_start_ff = [[c, 1, current_fire[c], 1, f_to, 0, tau[f_to, current_fire[c]], 0, 0]
                  for c=1:NUM_CREWS, f_to=1:NUM_FIRES if current_fire[c] != -1]
from_start_ff = copy(reduce(hcat, from_start_ff)')

18×9 Matrix{Int64}:
  3  1  1  1  1  0  1  0  0
  4  1  1  1  1  0  1  0  0
  5  1  4  1  1  0  1  0  0
  7  1  4  1  1  0  1  0  0
  8  1  4  1  1  0  1  0  0
 10  1  4  1  1  0  1  0  0
  3  1  1  1  2  0  1  0  0
  4  1  1  1  2  0  1  0  0
  5  1  4  1  2  0  1  0  0
  7  1  4  1  2  0  1  0  0
  8  1  4  1  2  0  1  0  0
 10  1  4  1  2  0  1  0  0
  3  1  1  1  3  0  1  0  0
  4  1  1  1  3  0  1  0  0
  5  1  4  1  3  0  1  0  0
  7  1  4  1  3  0  1  0  0
  8  1  4  1  3  0  1  0  0
 10  1  4  1  3  0  1  0  0

711×6 Matrix{Int64}:
  1  1  1   1   2  0
  2  1  1   1   2  0
  3  1  1   1   2  0
  4  1  1   1   2  0
  5  1  1   1   2  0
  6  1  1   1   2  0
  7  1  1   1   2  0
  8  1  1   1   2  0
  9  1  1   1   2  0
 10  1  1   1   2  0
  1  2  1   1   2  0
  2  2  1   1   2  0
  3  2  1   1   2  0
  ⋮                ⋮
  1  1  2  14  15  0
  2  1  2  14  15  0
  1  2  2  14  15  0
  2  2  2  14  15  0
  1  3  2  14  15  0
  2  3  2  14  15  0
  1  1  3  14  15  0
  2  1  3  14  15  0
  1  2  3  14  15  0
  2  2  3  14  15  0
  1  3  3  14  15  0
  2  3  3  14  15  0

In [1]:
struct RegionData
    
    crew_regions::Vector{Int64}
    fire_regions::Vector{Int64}
    
end

mutable struct RouteData
    
    routes_per_crew::Vector{Int64} # could add in length
    route_costs::Matrix{Real}
    
end

mutable struct SuppressionPlanData
    
    plans_per_fire::Vector{Int64} # could add in length
    plan_costs::Matrix{Real}
    
end

In [59]:
function ArcExitsRegion(from_type, from_ix, to_type, to_ix, region_data)
    
    # get the region where the arc originates
    from_region = 0
    if from_type == 1
        from_region = region_data.fire_regions[from_ix]
    elseif from_type == 2
        from_region = region_data.crew_regions[from_ix]
    else
        throw(DomainError(from_type, "from_type must be 1 or 2"))
    end
    
    # get the region where the arc terminates
    to_region = 0
    if to_type == 1
        to_region = region_data.fire_regions[to_ix]
    elseif to_type == 2
        to_region = region_data.crew_regions[to_ix]
    else
        throw(DomainError(from_type, "to_type must be 1 or 2"))
    end
    
    # if these are different regions
    if from_region != to_region
        
        # return the region that the arc exited
        return from_region
        
    end
    
    # otherwise
    return 0
    
end     

ArcExitsRegion (generic function with 1 method)

In [ ]:
struct ArcData
    
# crew, from_type, from_ix, to_type, to_ix, from_time, to_time, from_rested, to_rested, exited_region

In [ ]:
function get_out_of_region_stats(region, arcs_used, region_data)
    """
    """
    
    # restrict to the arcs that exited the given region
    out_of_region_ixs = [i for i in length(arcs_used[:, 1]) if arcs_used[i, 10] == region]
    out_of_region_arcs = arcs_used[out_of_region_ixs, :]
    
    # get the crews associated with this region
    crews = [i for i in 1:length(region_data.crew_regions) if region_data.crew_regions[i] == region]
    
    # initialize output array of indicator variables for crews exiting region
    out_array = zeros(length(crews), NUM_TIME_PERIODS)
    
    # for each crew in the region
    for i in 1:length(crews)
        
        # restrict to the arcs involving this crew
        crew_ixs = [j for j in length(out_of_region_arcs[:, 1]) if arcs_used[j, 1] == crews[i]]
        crew_arcs = out_of_region_arcs[crew_ixs, :]
        
        # get the times of rotation
        rotation_times = [t in crew_arcs[:, 6] ? 1 : 0 for t in 1:NUM_TIME_PERIODS]
        
        # update output for crew
        out_array[i, :] = rotation_times
        
    end
    
    return out_array

In [40]:
A = [1, "f", 1, "f", 2, 3, 4, 0, 0, 1]

10-element Vector{Any}:
 1
  "f"
 1
  "f"
 2
 3
 4
 0
 0
 1

In [39]:
[i == 1 ? 0 : 9 for i in 1:3]

3-element Vector{Int64}:
 0
 9
 9

In [33]:
A = [2, 3, 4, 5] .* [1, 2, 3]'

4×3 Matrix{Int64}:
 2   4   6
 3   6   9
 4   8  12
 5  10  15

In [35]:
A[:, 1] = [1, 2, 3, 4]
A

4×3 Matrix{Int64}:
 1   4   6
 2   6   9
 3   8  12
 4  10  15

In [30]:
A[[i for i in 1:length(A[:, 1]) if A[i, 1] <= 3], :]

2×4 Matrix{Int64}:
 2  4  6   8
 3  6  9  12

In [15]:
A[findall(A[:, 1] == 1)]

Int64[]

In [5]:
test = RouteData([6], zeros(4, 3))

RouteData([6], Real[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0])

In [9]:
push!(, 7)

2-element Vector{Int64}:
 6
 7

In [6]:
test.route_costs

4×3 Matrix{Real}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0